#4-7 ベイジアン学習

## 概要

**ベイジアン学習は、条件付き確率を使用した機械学習アルゴリズムで、ベイズの定理を利用して結果から原因を推論することを特徴としています。**

スパムメールフィルタや
ＥＣサイトのレコメンデーションなど、実社会の様々な場所に活用されています。

*   ベイズの定理
> ある条件A のもとで、事象Ｂが起こる確率（条件付き確率または事後確率）をP(B¦A)と表すと、事象B が起こったときに条件Ａであった事前確率P(A¦B)は、次のように表せます。これをベイズの定理といいます。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_07_01.jpg?raw=true" width="200px">

上の式は「原因の確率」を推算する式であり、ある結果を導く原因を推論するた
めに様々な用途で用いられています。

*   尤度
> あるデータが与えられたとき、どのような確率分布が最もよくデータの分布を表
すかをベイズの定理を用いて推定するとします。そのとき、確率分布の尤（もっと）
もらしさを表す指標を尤度といいます。
尤度は、尤度関数を用いて計算されます。
尤度関数は、条件付き確率と紐づいた関数のため、負の値はとらず、積分すると１
になります。ただし、確率密度関数とは別の概念です。尤度関数を最大化するパラメー
タを推定する手法を最尤法（Method of maximum likelihood）といいます。最尤法は1922 年にロナルド・フィッシャーが論文で初めて使用した用語です。


*   ナイーブベイズ分類器（ベイジアンフィルタ）
> ナイーブベイズモデルは、別名単純ベイズモデルと呼ばれ、事象同士が独立であると仮定した条件付き確率のモデルです。ナイーブベイズ分類器は、シンプルで処理が高速なので、文書分類やメールのスパムフィルタなどに広く用いられています。
一方で、単純なモデルで単語間の意味関係は処理できないため、精度が高くないとも指摘されています。

*   ベイジアンネットワーク
> ベイジアンネットワークは、原因と結果の複数の組み合わせを有向グラフで可視化した確率モデル（グラフィカルモデル）の一つです。1985年にジュディア・パールによって命名されました。パールは、この功績によりチューリング賞を受賞しています。

下図に単純なベイジアンネットワークの例を示します。ベイジアンネットワークは、各ノードがマルコフ性を満たす、つまり各ノードの状態が条件付き独立であることで、計算を大幅に簡略化できます。
ベイジアンネットワークは様々な原因と結果を推測することが可能であり、主観的な情報も対象にできるため、適用範囲が広いアルゴリズムです。クリストファー・
ビショップによる書籍などによって認知が広まったことと、コンピュータの計算能力向上により多数のノードを持つベイジアンネットワークの確率推論が可能になり、近年研究と活用が活発に進んでいます。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_07_02.jpg?raw=true" width="650px">



##ナイーブベイズクラス分類器の実装
ナイーブベイズモデルは非常に高速で簡単な分類アルゴリズムの1 つであり、非常に高次元の
データセットに適しています。このアルゴリズムは非常に高速で、チューニング可能なパラメータ
がとても少ないため、分類問題の最初の一手として非常に役立ちます。

ナイーブベイズ分類器がどのように動作するかを直感的に説明し、いくつかのデータセットに対する例を示します。

###ガウシアンナイーブベイズ
ナイーブベイズ分類器（Naive Bayes classifiers）はベイズ分類法に基づいて作られています。これは統計量に対する条件付き確率の関係を説明するベイズの定理に依存しています。

ベイズ分類で
は、ある特徴量（features）が観測された際に、そのラベルがL である確率に興味があります。これをP(L | features)と記述します。ベイズの定理は、これを直接的に計算できる量から求める方法を与えます。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_07_03.jpg?raw=true" width="360px">

2 つのラベル（ここではL1とL2としましょう）を決定することを考えます。1 つの方法は各ラベルの事後確率の比を計算することです。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_07_04.jpg?raw=true" width="360px">

各ラベルに対するP(features | Li)を計算するためのモデルが必要です。このようなモデルは、データを生成する仮想的なランダムプロセスを指定するため、生成モデル（generative model）と呼ばれます。
各ラベルに対してこの生成モデルを指定することが、ベイズ分類器学習の主要な部分となります。

一般化された学習を行うのは非常に難しい作業なのですが、いくつかのモデルに対する単純な前提を取り入れることで簡単に行うことができます。
これが「ナイーブベイズ」の「ナイーブ」（単純）の理由です。各ラベルの生成モデルについて非常に単純な仮定をすると、各クラスに対する生成モデルのおおよその近似を使って、ベイズ分類が行えます。さまざまなタイプのナイーブベイズ分類器は、データに関するそれぞれの単純な仮定に基づいています。以下の節でいくつかの例を見てみましょう。まず標準的なモジュールをインポートします。

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

ナイーブベイズ分類器の中でおそらく最も単純なものは、ガウシアンナイーブベイズ分類器でしょう。この分類器では、各ラベルからのデータが単純なガウス分布に従うという仮定に基づいています。次のデータで考えてみましょう。

In [0]:
from sklearn.datasets import make_blobs
X, y = make_blobs(100, 2, centers=2, random_state=2, cluster_std=1.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu');

データが次元間の共分散を伴わないガウス分布に基づいていると仮定することで、非常に簡単に
モデルが作成できます。モデルへの当てはめを行うには、各ラベルに対するデータポイントの平均
と標準偏差を見つけるだけで可能となります。分布を定義するのに値は、その2 つだけだからです。
この単純なガウス推定の結果を下図に示します。

In [0]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X, y)
#model.predict(X)

In [0]:
from sklearn.datasets import make_blobs
X, y = make_blobs(100, 2, centers=2, random_state=2, cluster_std=1.5)

fig, ax = plt.subplots()

ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu')
ax.set_title('Naive Bayes Model', size=14)

xlim = (-8, 8)
ylim = (-15, 5)

xg = np.linspace(xlim[0], xlim[1], 60)
yg = np.linspace(ylim[0], ylim[1], 40)
xx, yy = np.meshgrid(xg, yg)
Xgrid = np.vstack([xx.ravel(), yy.ravel()]).T

for label, color in enumerate(['red', 'blue']):
    mask = (y == label)
    mu, std = X[mask].mean(0), X[mask].std(0)
    P = np.exp(-0.5 * (Xgrid - mu) ** 2 / std ** 2).prod(1)
    Pm = np.ma.masked_array(P, P < 0.03)
    ax.pcolorfast(xg, yg, Pm.reshape(xx.shape), alpha=0.5,
                  cmap=color.title() + 's')
    ax.contour(xx, yy, P.reshape(xx.shape),
               levels=[0.01, 0.1, 0.5, 0.9],
               colors=color, alpha=0.2)
    
ax.set(xlim=xlim, ylim=ylim)

#fig.savefig('figures/05.05-gaussian-NB.png')

この楕円は、各ラベルに対するガウシアン生成モデルを表し、楕円の中心に向かうほど高い確率を示します。この生成モデルをクラスごとに配置することで、任意のデータポイントに対する尤度P(features | L1)を計算する簡単な手法が得られます。そのため、事後確率比を簡単に計算し、各データポイントのラベルが何であるのが最も確からしいかを判定できます。

この手順はscikit-learnのsklearn.naive_bayes.GaussianNB 推定器で実装されています。

新しいデータを生成し、ラベルを予測してみましょう。

In [0]:
rng = np.random.RandomState(0)
Xnew = [-6, -14] + [14, 18] * rng.rand(2000, 2)
ynew = model.predict(Xnew)

この新しいデータをプロットして、境界がどこにあるのかを可視化できます

In [0]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu')
lim = plt.axis()
plt.scatter(Xnew[:, 0], Xnew[:, 1], c=ynew, s=20, cmap='RdBu', alpha=0.1)
plt.axis(lim);

境界はわずかに湾曲しています。一般に、ガウシアンナイーブベイズの境界は2 次曲線となります。

ベイズ主義の良いところは、確率的な分類が可能である点です。これはpredict_proba メソッドを使って計算できます。

In [0]:
yprob = model.predict_proba(Xnew)
yprob[-8:].round(2)

結果の列は、それぞれ第1 および第2 ラベルに対する事後確率を表します。分類の不確実性についての推定値を求めたいのであれば、このようなベイズのアプローチは非常に有用です。

もちろん、分類の最終的な良し悪しは、モデルに対して行った仮定の良し悪しに依存します。ガウシアンナイーブベイズの結果がそれほど良いものとならない理由がここにあります。それでも、多くの場合、特に特徴の数が多くなるにつれて、こうした単純な仮定がガウシアンナイーブベイズの有用性を高めています。

###多項分布ナイーブベイズ
前の例で説明したガウスの仮定は、各ラベルの生成分布を指定する際に使用できる単純な仮定の1つにすぎません。もう1 つの有用な例は、単純な多項分布から生成されると仮定した多項分布ナ
イーブベイズ（Multinomial Naive Bayes）です。多項分布は、複数のカテゴリがどれだけ観測されるかの確率を表しているため、多項分布ナイーブベイズは出現数または出現レートレートを表す特徴量に最も適しています。

最良のガウス分布を持つデータ分布をモデル化するのではなく、最適な多項分布を用いてデータ分布をモデル化するという点を除けば、考え方は前の例とまったく同じです。

多項分布ナイーブベイズが頻繁に使用される問題の1 つが、文書内の語数または単語の出現頻度に関連するテキスト分類です。

ここでは、20個のニュースグループのコーパスから得られる単語数を用い、短い文書をカテゴリに分類する方法を示します。
データをダウンロードして、ニュースグループ名を見てみましょう。

In [0]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups()
data.target_names

簡略化のために、対象のカテゴリを絞り、学習セットとテストセットをダウンロードします。

In [0]:
categories = ['talk.religion.misc', 'soc.religion.christian', 'sci.space',
'comp.graphics']
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

代表的なデータを1 つ見てみましょう。

In [0]:
print(train.data[5])

このデータを機械学習で使用するには、各データの内容を数値のベクトルに変換する必要があります。このために、TF-IDFベクトル化器を使用し、多項分布ナイーブベイズ分類器に接続するパイプラインを作成します。

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

このパイプラインを使用して、モデルを学習データに適用し、テストデータのラベルを予測できます。

In [0]:
model.fit(train.data, train.target)
labels = model.predict(test.data)

テストデータのラベルを予測したので、推定値の評価が行えます。例えば、テストデータの真の
ラベルと予測したラベルの間の混同行列を次に示します。

In [0]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(test.target, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
xticklabels=train.target_names, yticklabels=train.target_names, cmap='RdPu')
plt.xlabel('true label')
plt.ylabel('predicted label')

確かに、この非常に単純な分類器であっても、宇宙に関する話題とコンピュータに関する話題を
うまく分類できましたが、宗教に関する話題とキリスト教に関する話題では混乱しました。おそら
くこれは想定内の混乱です。

素晴らしいことに、このパイプラインのpredict() メソッドを使用すれば、任意の文字列のカテゴリを決定するツールになります。ここでは、一連の文字列に対する分類予測を返す簡単なユーティリティ関数を示します。

In [0]:
def predict_category(s, train=train, model=model):
  pred = model.predict([s])
  return train.target_names[pred[0]]

いくつか試してみましょう。

In [0]:
predict_category('sending a payload to the ISS')

In [0]:
predict_category('discussing islam vs atheism')

In [0]:
predict_category('determining the screen resolution')

これは、文字列内の各単語の（重み付けされた）頻度による単純な確率モデルと比べて、それほど洗練されたものではないことを覚えておいてください。

それにもかかわらず、得られる結果は印象的です。非常に単純なアルゴリズムであっても、慎重に使用し、高次元の大きなデータセットで
学習したならその結果は驚くほど効果的となります。

ナイーブベイズ分類器はデータ対して単純な仮定をするため、複雑なモデルと同じようには機能しません。とは言うものの、いくつかの利点があります。
*   学習と予測のどちらも非常に高速です
*   わかりやすい確率的予測を提供します
*   多くの場合で、非常に解釈が簡単です
*   調整可能なパラメータが非常に少数です

れらの利点は、最初に使用する分類器として、ナイーブベイズ分類器が良い選択であることを意味します。それがうまく働くのであれば、非常に高速で非常にわかりやすい分類器が得られたこ
とになります。うまくいかない場合は、より洗練されたモデルの調査を始めましょう。どうなれば良いのか、その基準とすべき知識は、ナイーブベイズ分類器の結果から既に得られています。

ナイーブベイズ分類器は、次のような状況に対して特に優れています。
*   単純な仮定がデータと一致する場合（実際には非常にまれです）
*   モデルの複雑性がそれほど重要ではなく、カテゴリが非常によく分離されている場合
*   モデルの複雑性がそれほど重要ではなく、データの次元が非常に高い場合

最後の2つは異なるように見えますが、実際には関連しています。データセットの次元が拡大するにつれて、2つの点が近くにある可能性は低くなります（結局、2 つの点がすべての次元で近くになければ、総合的に2 つの点が近傍にあることにはなりません）。

これは、新しい次元には情報が追加されていると仮定するなら、高次元のクラスタは低次元のクラスタよりも平均的に離散する傾向があることを意味します。この理由から、ナイーブベイズのように単純な分類器は、次元が増えるにつれてより複雑な分類器よりもうまく機能する傾向があります。データが十分であれば、単純なモデルでも強力になります。







